### WSI zad 2

Implementacja algorytmu ewolucyjnego zawierająca

*  selekcję turniejową (k=2)
*  krzyżowanie jednopunktowe
*  mutacja gaussowska
*  sukcesja generacyjna





In [16]:
import numpy as np
import random
import matplotlib.pyplot as plt
def f1(x1, y1):
    return (x1**2 + y1 - 11)**2 + (x1 + y1**2 - 7)**2

def f2(x2, y2):
    return 2 * x2**2 + 1.05 * x2**4 + x2**6/6 + x2*y2 + y2**2

population_size = 100
number_of_it = 201
mutation_rate = 0.05
mutation_strength = 0.01
tournament_size = 2;
crossover_rate = 0.85

# Creating a population
def initialize_population(population_size):
  population = []
  for _ in range(population_size):
      x1 = random.uniform(-5, 5)
      y1 = random.uniform(-5, 5)
      x2 = random.uniform(-5, 5)
      y2 = random.uniform(-5, 5)
      population.append((x1, y1, x2, y2))
  return population


def initialize_second_population(population_size):
  population = []
  for _ in range(population_size):
      x1 = random.normalvariate(-0.3,1)
      x1 = max(-5, min(5, x1))
      y1 = random.normalvariate(-0.9,1)
      y1 = max(-5, min(5, y1))
      x2 = random.uniform(-5, 5)
      y2 = random.uniform(-5, 5)
      population.append((x1, y1, x2, y2))
  return population

# Tournament selection
def tournament_selection(population, tournament_size):
  selected = []
  while len(selected) < len(population):
      tournament = random.sample(population, tournament_size)
      winner = min(tournament, key=lambda ind: f1(ind[0], ind[1]) + f2(ind[2], ind[3]))
      selected.append(winner)
  return selected

# one-point crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(1, 3)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# gauss-mutation
def mutate(individual, mutation_rate):
    mutated = []
    for gene in individual:
        if random.random() < mutation_rate:
            mutation = np.random.normal(0, 1)*mutation_strength
            mutated.append(gene + mutation)
        else:
            mutated.append(gene)
    return tuple(mutated)

# elite succession
def generational_succession(population, offspring):
    new_population = sorted(population + offspring, key=lambda ind: f1(ind[0], ind[1]) + f2(ind[2], ind[3]) )
    return new_population[:population_size]

def visualize_population(population, generation, text):
    x1_values = [ind[0] for ind in population]
    y1_values = [ind[1] for ind in population]
    x2_values = [ind[2] for ind in population]
    y2_values = [ind[3] for ind in population]

    plt.figure(figsize=(8, 8))

    plt.subplot(2, 2, 1)
    plt.scatter(x1_values, y1_values, s=20, alpha=0.6)
    plt.title(f'Generation {generation} - Population Distribution (f1)\n {text}')
    plt.xlabel('x1')
    plt.ylabel('y1')
    plt.subplot(2, 2, 2)
    plt.scatter(x2_values, y2_values, s=20, alpha=0.6)
    plt.title(f'Generation {generation} - Population Distribution (f2)\n {text}')
    plt.xlabel('x2')
    plt.ylabel('y2')

    plt.tight_layout()
    plt.show()

# def visualize_population_one_point(population, generation, random_point):
#     plt.figure(figsize=(8,8))
#     plt.subplot(2, 2, 1)
#     plt.scatter(random_point[0],random_point[1])
#     plt.title(f'Random individual travel')
#     plt.xlabel('x1')
#     plt.ylabel('y1')
#     plt.subplot(2, 2, 2)
#     plt.scatter(random_point[2],random_point[3])
#     plt.title(f'Random individual travel')
#     plt.xlabel('x2')
#     plt.ylabel('y2')

#     plt.tight_layout()
#     plt.show()


# main algorithm
population = initialize_population(population_size)

for generation in range(number_of_it):
    # random_point = population[0]
    # if (generation % 10 == 0 or generation < 10):
    #     visualize_population(population, generation,'')
        # visualize_population_one_point(population, generation, random_point)
    selected = tournament_selection(population, tournament_size)
    offspring = []
    while len(offspring) < population_size:
        parent1, parent2 = random.sample(selected, 2)
        if random.random() < crossover_rate:
          child1,child2 = crossover(parent1, parent2)
        else:
          child1, child2 = parent1, parent2
        child1 = mutate(child1, mutation_rate)
        child2 = mutate(child2, mutation_rate)
        offspring.append(child1)
        offspring.append(child2)
    # if (generation % 10 == 0 or generation < 10):
    #   visualize_population(offspring, generation,'with crossover')
    # generational succession
    population = offspring
    # population = generational_succession(population, offspring)


# best
best_individual = min(population, key=lambda ind: f1(ind[0], ind[1]) + f2(ind[2], ind[3]))

best_fitness_f1 = f1(best_individual[0], best_individual[1])
best_fitness_f2 =  f2(best_individual[2], best_individual[3])
best_fitness = best_fitness_f1+best_fitness_f2
print("Best individual:", best_individual)
print("Best fitness of f1:", best_fitness_f1)
print("Best fitness of f2:", best_fitness_f2)
print("Best fitness of sum:", best_fitness)

Best individual: (3.583978551102494, -1.8490212926897436, -0.02608264383849472, 0.04783342384991313)
Best fitness of f1: 2.513630615864674e-05
Best fitness of f2: 0.0024015089056910015
Best fitness of sum: 0.0024266452118496483


In [17]:
def evaluate_fitness(individual, function_num):
    x1, y1, x2, y2 = individual
    if function_num == 1:
        return f1(x1, y1)
    else:
        return f2(x2, y2)

population_size = 100
number_of_it = 201
mutation_rate = 0.05
mutation_strength = 0.01
tournament_size = 2;
crossover_rate = 0.85

best_fitness_f1_avg = 0
best_fitness_f2_avg = 0
best_fitness_avg = 0
best_individual_f1 = None
best_individual_f2 = None

num_runs = 50


for run in range(num_runs):
  for generation in range(number_of_it):
      # random_point = population[0]
      # if (generation % 10 == 0 or generation < 10):
      #     visualize_population(population, generation,'')
          # visualize_population_one_point(population, generation, random_point)
      selected = tournament_selection(population, tournament_size)
      offspring = []
      while len(offspring) < population_size:
          parent1, parent2 = random.sample(selected, 2)
          if random.random() < crossover_rate:
            child1,child2 = crossover(parent1, parent2)
          else:
            child1, child2 = parent1, parent2
          child1 = mutate(child1, mutation_rate)
          child2 = mutate(child2, mutation_rate)
          offspring.append(child1)
          offspring.append(child2)
      # if (generation % 10 == 0 or generation < 10):
      #   visualize_population(offspring, generation,'with crossover')
      # generational succession
      population = offspring
      # population = generational_succession(population, offspring)

  # evaluating the individuals
  best_individual_run = min(population, key=lambda ind: f1(ind[0], ind[1]) + f2(ind[2], ind[3]))

  # best_individual_f1_run = min(population, key=lambda ind: f1(ind[0], ind[1]))
  best_fitness_f1_run = f1(best_individual_run[0], best_individual_run[1])

  # best_individual_f2_run = min(population, key=lambda ind: f2(ind[2], ind[3]))
  best_fitness_f2_run = f2(best_individual_run[2], best_individual_run[3])

  best_fitness_f1_avg += best_fitness_f1_run
  best_fitness_f2_avg += best_fitness_f2_run

  if best_individual is None or best_fitness_f1_run < best_fitness_f1:
      best_individual = best_individual_run

best_fitness_f1_avg /= num_runs
best_fitness_f2_avg /= num_runs

print("Best individuals:", best_individual)
print(f'Best fitness for f1 (average over {num_runs} runs):', best_fitness_f1_avg)
print(f'Best fitness for f2 (average over {num_runs} runs):', best_fitness_f2_avg)
print(best_fitness_f1_avg+best_fitness_f2_avg)

Best individuals: (3.584424524431762, -1.8481260330365814, 1.0158354952660643e-06, -2.5988645683434422e-06)
Best fitness for f1 (average over 50 runs): 1.3970919228092227e-09
Best fitness for f2 (average over 50 runs): 1.08558722961934e-09
2.4826791524285627e-09
